# Libraries used

Running Kernel3.9.13 base anaconda

pip install squarify
pip install yellowbrick
pip install plotly
pip install seaborn
pip install lazypredict
pip install pandas_profiling

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
#import squarify #treemap
import os
import matplotlib
import warnings

from imblearn.over_sampling import SMOTE


#to enable the inline plotting
%matplotlib inline 


import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [10]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#rfc using regularization and gridsearch to find the best parameters for the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier


warnings.simplefilter(action='ignore', category=FutureWarning)

ImportError: cannot import name 'DataError' from 'pandas.core.base' (c:\Users\roser\anaconda3\lib\site-packages\pandas\core\base.py)

In [11]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df.head(2)

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

def outlier_function(df, col_name):
    """ this function detects first and third quartile and interquartile range for a given column of a dataframe
    then calculates upper and lower limits to determine outliers conservatively
    returns the number of lower and uper limit and number of outliers respectively"""
    first_quartile = np.percentile(np.array(df[col_name].tolist()), 25)
    third_quartile = np.percentile(np.array(df[col_name].tolist()), 75)
    IQR = third_quartile - first_quartile
                        
    upper_limit = third_quartile+(3*IQR)
    lower_limit = first_quartile-(3*IQR)
    outlier_count = 0
                    
    for value in df[col_name].tolist():
        if (value < lower_limit) | (value > upper_limit):
            outlier_count +=1
        else:
            pass
    return lower_limit, upper_limit, outlier_count

In [12]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#display all columns
pd.set_option('display.max_columns', None)


# Importing previous dataset

In [23]:
#point to the folder where the data is stored
#os.chdir(r"C:\Users\pedro\datathon")

# Loading combined_mod dataset
train_consolidated = pd.read_csv('train_consolidated_with_outliers.csv')
sample_submission = pd.read_csv('submission_sample_prepared2.csv')

In [14]:
train_consolidated = train_consolidated[sample_submission.columns]

KeyError: "['Diameter', 'Length', 'Pressure', 'aspect', 'No_Incidences_Total'] not in index"

In [24]:
#input null values in sample_submission with mean of the column
sample_submission['Yearly Sun Hours (hours)'] = sample_submission['Yearly Sun Hours (hours)'].fillna(sample_submission['Yearly Sun Hours (hours)'].mean())

In [25]:
train_consolidated.head(1)

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,No_Inspections,No_Previous_Incidences,average_severity_pipe,relative_risk,preventive_maintenance_rate,Probability_rate,Average_MonthsLastRev,pipe_inspected_frequently,Age_pipe_at_inspection,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,incidence_area,connection_bool,Severity_high,Severity_medium,Severity_low,No_Previous_Severity_Low,No_Previous_Severity_Medium,No_Previous_Severity_High,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,489616,ZRV-00003385,2013,2013-05-08,16,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,1,0.0,4.0,0.0,0.0,0.0,21.4,1,1,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5


In [26]:
#creating a subset for altering the dataset after initial EDA
df_baseline = train_consolidated.copy()


In [27]:
df_baseline.head(1)

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,No_Inspections,No_Previous_Incidences,average_severity_pipe,relative_risk,preventive_maintenance_rate,Probability_rate,Average_MonthsLastRev,pipe_inspected_frequently,Age_pipe_at_inspection,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,incidence_area,connection_bool,Severity_high,Severity_medium,Severity_low,No_Previous_Severity_Low,No_Previous_Severity_Medium,No_Previous_Severity_High,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,489616,ZRV-00003385,2013,2013-05-08,16,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,1,0.0,4.0,0.0,0.0,0.0,21.4,1,1,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5


In [28]:
#count values for Incidence
df_baseline['Incidence'].value_counts()


0    6324136
1      12084
Name: Incidence, dtype: int64

# Approach 1 : Stratifying with undersampling 


In [29]:
df_baseline_undersampling = df_baseline.groupby('Incidence').apply(lambda x: x.sample(df_baseline['Incidence'].value_counts().min(), random_state=42)).reset_index(drop=True)

df_baseline_undersampling['Incidence'].value_counts()

0    12084
1    12084
Name: Incidence, dtype: int64

In [30]:
#do a train test split to have a test set to evaluate the model
from sklearn.model_selection import train_test_split

X = df_baseline_undersampling.drop('Incidence', axis=1)
y = df_baseline_undersampling['Incidence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
df_baseline.head()

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,No_Inspections,No_Previous_Incidences,average_severity_pipe,relative_risk,preventive_maintenance_rate,Probability_rate,Average_MonthsLastRev,pipe_inspected_frequently,Age_pipe_at_inspection,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,incidence_area,connection_bool,Severity_high,Severity_medium,Severity_low,No_Previous_Severity_Low,No_Previous_Severity_Medium,No_Previous_Severity_High,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,489616,ZRV-00003385,2013,2013-05-08,16,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,1,0.0,4.0,0.0,0.0,0.0,21.4,1,1,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5
1,489616,ZRV-00003385,2015,2015-03-04,22,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,2,0.0,4.0,0.0,0.0,0.0,21.4,1,3,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5
2,489616,ZRV-00003385,2016,2017-01-12,22,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,3,0.0,4.0,0.0,0.0,0.0,21.4,1,4,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5
3,489616,ZRV-00003385,2018,2018-12-19,23,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,4,0.0,4.0,0.0,0.0,0.0,21.4,1,6,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5
4,489616,ZRV-00003385,2020,2020-12-05,24,0,ciudad real,Torralba de Calatrava,2012,0.063,7.792,0.4,0,0,0,5,0.0,4.0,0.0,0.0,0.0,21.4,1,8,49.473016,0.1575,1.542199,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.003969,60.715264,0.16,15.441667,1.5,33.7,438,0.545833,52,107.5


# Approach 2 :  SMOTE to tackle the unbalanced dataset problem

#using SMOTE to balance the dataset creating a subset dataset with target y = 'Incidence' and removing Incidence','MonthsLastRev','InspectionDay','PipeId', 'MaintenanceId', 'InspectionYear', 'InspectionDate',  'Province', 'Town', 'YearBuilt' for x
X = df_baseline.drop(['PipeId','Province', 'Town'], axis=1)
y = df_baseline['Incidence']
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_res.shape,y_res.shape

#create a pandas dataframe with the new balanced dataset
df_baseline_balanced = pd.DataFrame(X_res, columns=X.columns)
df_baseline_balanced['Incidence'] = y_res
df_baseline_balanced.head()


In [31]:
#using SMOTE to balance the dataset creating a subset dataset with target y = 'Incidence'
X = df_baseline.drop('Incidence', axis=1)
y = df_baseline['Incidence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

#count values for Incidence in y_train
y_train.value_counts()



ValueError: could not convert string to float: 'ZRV-00003532'

In [ ]:
y_train.head(1)

0    0.0
Name: Incidence, dtype: float64

In [ ]:
df_baseline.drop['Main']

# Hyperparameter approach


## XGBoost classifier

In [ ]:
# Create a dictionary of hyperparameters and their values
params = {
    'eta': [0.1],
    'gamma': [0.001, 0.1],
    'max_depth': [10, 50],
    'min_child_weight': [3, 10],
    'subsample': [0.5, 1],
    'colsample_bytree': [0.1, 1],
    'num_parallel_tree': [1, 100]
}

# Create an XGBoost model object
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')

# Create a GridSearchCV object with the XGBoost model and hyperparameters
grid_search = GridSearchCV(xgb_model, params, scoring='roc_auc', cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the test data
xgb_model_best = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', **grid_search.best_params_)
xgb_model_best.fit(X_train, y_train)
y_pred_prob = xgb_model_best.predict_proba(X_test)[:, 1]
print("Test AUC:", roc_auc_score(y_test, y_pred_prob))

In [ ]:
#use xgboost classifier to predict the probability of Incidence in train and test sets using cross validation to get best parameters
xgb = XGBClassifier(random_state=42, n_gpu=1, tree_method='gpu_hist', gpu_id=0, verbosity=1 ,eta=0.1 objective='binary:logistic', eval_metric='auc', predictor='gpu_predictor')
params = {'learning_rate': [0.05, 0.1, 0.3],
          'max_depth': [3, 4, 5],
          'n_estimators': [10, 50, 100, 200],
          'subsample': [0.5, 0.7, ]
          }
xgb_cv = GridSearchCV(xgb, param_grid=params, scoring='roc_auc', cv=5, n_jobs=-1, verbose=1)
xgb_cv.fit(X_train, y_train)
print("Best parameters: {}".format(xgb_cv.best_params_))
print("Best cross-validation score: {:.2f}".format(xgb_cv.best_score_))
print("Test set AUC: {:.2f}".format(roc_auc_score(y_test, xgb_cv.predict_proba(X_test)[:, 1])))


In [ ]:
predictions = xgb_cv.predict_proba(X_test)
predictions = predictions[:, 1]

#show accuracy score
accuracy = accuracy_score(y_test, predictions.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#show confusion matrix
matrix = confusion_matrix(y_test, predictions.round())
print(matrix)


In [ ]:

#show classification report
report = classification_report(y_test, predictions.round())
print(report)



In [ ]:
#drop incidence in sample_submission
sample_submission = sample_submission.drop('Incidence', axis=1)

In [ ]:
#make predictions for submission using the model
predictions = xgb_cv.predict_proba(sample_submission)
predictions = predictions[:, 1]



In [ ]:
#add predictions to submission
sample_submission ['Incidence'] = predictions

#show submission
sample_submission.head(1)


In [ ]:
#drop all columns except PipeId and Incidence in submission2 creating kaggle_submission
kaggle_submission = sample_submission.drop(['Diameter', 'Length', 'Pressure', 'NumConnections', 'aspect',
       'Relative_Thickness', 'area_connection', 'gas_natural',
       'Material_Acrylonitrile-Butadiene-Styrene', 'Material_Copper',
       'Material_Fiberglass-Reinforced Plastic', 'Material_Polyethylene',
       'Material_Polypropylene', 'Yearly Sun Hours (hours)',
       'Average_MonthsLastRev', 'MonthsLastRev', 'No_Inspections',
       'No_Incidences_Total', 'Age_pipe_at_inspection',
       'Population density (persons/sqkm)'], axis=1)



In [ ]:
#show kaggle_submission
kaggle_submission.columns


In [ ]:

kaggle_submission.shape


In [ ]:
kaggle_submission.head(1)
#export kaggle_submission to csv
os.chdir(r"C:\Users\pedro\datathon")

kaggle_submission.to_csv('kaggle_submission_xgb_new_features.csv', index=False)


In [ ]:
kaggle_submission.head(1)
